In [1]:
import torch

In [2]:
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25, 300)
b = torch.ones(22, 300)
c = torch.ones(15, 300)
pad_sequence([a, b, c]).size()

torch.Size([25, 3, 300])

In [6]:
a = torch.ones(3, 10)
b = torch.ones(2, 10)
c = torch.ones(4, 10)
pad_sequence([a, b, c], batch_first=True, padding_value=-1)

tensor([[[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]],

        [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]],

        [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]])

In [10]:
torch.tensor([[1,2,3]]).shape

torch.Size([1, 3])

In [17]:
def fn(item: int or str):
    if isinstance(item, str):
        print("str")
    elif isinstance(item, int):
        print("int")
    else:
        raise ValueError("wrong")

In [18]:
fn(12.5)

ValueError: wrong

In [19]:
from collections import OrderedDict

In [21]:
d = OrderedDict({
    'a': 2, 'b': 3})

In [22]:
d

OrderedDict([('a', 2), ('b', 3)])

In [26]:
d['b']

3

In [27]:
d.keys()

odict_keys(['a', 'b'])

In [28]:
d.insert

AttributeError: 'collections.OrderedDict' object has no attribute 'insert'

In [48]:
voc_list = ['x', 'y', 'w']
dx_map = {token: i for i, token in enumerate(voc_list)}

In [49]:
dx_map

{'x': 0, 'y': 1, 'w': 2}

In [44]:
voc_list.insert(-1, 'a')
voc_list

['x', 'y', 'a', 'w']

In [50]:
voc_list.insert(2, 'a')
voc_list

['x', 'y', 'a', 'w']

In [47]:
voc_list.insert(3, 'a')
voc_list

['x', 'y', 'w', 'a']

In [51]:
voc_list[2:2] = "f"

In [52]:
voc_list

['x', 'y', 'f', 'a', 'w']

In [53]:
voc_list[2:2] = ["g", "t"]

In [54]:
voc_list

['x', 'y', 'g', 't', 'f', 'a', 'w']

In [55]:
voc_list[2] = ["g", "t"]

In [56]:
voc_list

['x', 'y', ['g', 't'], 't', 'f', 'a', 'w']

In [59]:
    DATASET_URLS = OrderedDict({
        'train': "train.{lan}.gz",
        'test': 'test_2016_flickr.{lan}.gz',
        'val': "val.{lan}.gz",
    })

In [60]:
DATASET_URLS

OrderedDict([('train', 'train.{lan}.gz'),
             ('test', 'test_2016_flickr.{lan}.gz'),
             ('val', 'val.{lan}.gz')])

In [61]:
OrderedDict([('train', [['/home/steven/proj/lifematrix/TransformerLM/.data/train.de'], ['/home/steven/proj/lifematrix/TransformerLM/.data/train.en']]), ('val', [['/home/steven/proj/lifematrix/TransformerLM/.data/val.de'], ['/home/steven/proj/lifematrix/TransformerLM/.data/val.en']]), ('test', [['/home/steven/proj/lifematrix/TransformerLM/.data/test_2016_flickr.de'], ['/home/steven/proj/lifematrix/TransformerLM/.data/test_2016_flickr.en']])])


OrderedDict([('train',
              [['/home/steven/proj/lifematrix/TransformerLM/.data/train.de'],
               ['/home/steven/proj/lifematrix/TransformerLM/.data/train.en']]),
             ('val',
              [['/home/steven/proj/lifematrix/TransformerLM/.data/val.de'],
               ['/home/steven/proj/lifematrix/TransformerLM/.data/val.en']]),
             ('test',
              [['/home/steven/proj/lifematrix/TransformerLM/.data/test_2016_flickr.de'],
               ['/home/steven/proj/lifematrix/TransformerLM/.data/test_2016_flickr.en']])])

In [63]:
import math

class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start, "this example code only works with end >= start"
        self.start = start
        self.end = end
    def __iter__(self):
        return iter(range(self.start, self.end))
# should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
ds = MyIterableDataset(start=3, end=7)

# Single-process loading
print(list(torch.utils.data.DataLoader(ds, num_workers=0)))
# Directly doing multi-process loading yields duplicate data
print(list(torch.utils.data.DataLoader(ds, num_workers=2)))

# Define a `worker_init_fn` that configures each dataset copy differently
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset  # the dataset copy in this worker process
    overall_start = dataset.start
    overall_end = dataset.end
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker, overall_end)

# Mult-process loading with the custom `worker_init_fn`
# Worker 0 fetched [3, 4].  Worker 1 fetched [5, 6].
print(list(torch.utils.data.DataLoader(ds, num_workers=2, worker_init_fn=worker_init_fn)))

# With even more workers
print(list(torch.utils.data.DataLoader(ds, num_workers=12, worker_init_fn=worker_init_fn)))

[tensor([3]), tensor([4]), tensor([5]), tensor([6])]
[tensor([3]), tensor([3]), tensor([4]), tensor([4]), tensor([5]), tensor([5]), tensor([6]), tensor([6])]
[tensor([3]), tensor([5]), tensor([4]), tensor([6])]
[tensor([3]), tensor([4]), tensor([5]), tensor([6])]


In [65]:
len(ds)

TypeError: object of type 'MyIterableDataset' has no len()

In [67]:
from functools import partial

def fn(a, b):
    print(a, b)
    
fn1 = partial(fn, 10)
fn1(5)

10 5


In [80]:
from dotteddict import dotteddict
import yaml

In [73]:
C = dotteddict(
{ 'a': 1,
  'b': 2,
  'd': {'x':1, 'y': 2}
}
)

In [74]:
C

{'a': 1, 'b': 2, 'd': {'x': 1, 'y': 2}}

In [75]:
C.a

1

In [76]:
C.b

2

In [79]:
C.d.y

2

In [85]:
yaml.load(
"""
a: 1
b: 2
c:
   d: 100
   e: 200
"""
,
yaml.SafeLoader)

{'a': 1, 'b': 2, 'c': {'d': 100, 'e': 200}}

In [ ]:
yaml.Safe

In [86]:
C = dotteddict(
yaml.load(
"""
a: 1
b: 2
c:
   d: 100
   e: 200
"""
,
yaml.SafeLoader)
)

In [88]:
C

{'a': 1, 'b': 2, 'c': {'d': 100, 'e': 200}}

In [89]:
C.c.d

100

In [39]:
import torch
import numpy as np

In [20]:
scores = torch.randn([16,8,20,20])
mask = torch.randint(2, [20,20], dtype=torch.bool)
# mask[1,:] = 1.0

In [26]:
repeated_mask = mask.repeat(16, 8,1,1)

In [27]:
check = torch.tensor([torch.all(mask == repeated_mask[i, j]) for i in range(16) for j in range(8)])
torch.all(check)
        

tensor(True)

In [33]:
c1 = scores.masked_fill(mask==0, -1e9)
c2 = scores.masked_fill(repeated_mask==0, -1e9)

In [34]:
torch.all(c1==c2)

tensor(True)

In [38]:
mask.repeat(1,1).shape

torch.Size([20, 20])

In [40]:
a = np.ones((3,4))
a.shape

(3, 4)

In [43]:
np.tile(a, [2,2, 1, 1]).shape

(2, 2, 3, 4)

In [46]:
scores = torch.randn([16,8,26,20])
mask = torch.randint(2, [1, 20], dtype=torch.bool)
scores.masked_fill_(mask==0, -1e9)

tensor([[[[ 1.1300e+00,  8.6699e-01, -1.0000e+09,  ..., -3.1115e-01,
           -1.0000e+09, -1.0000e+09],
          [-3.5055e-01, -3.1816e-01, -1.0000e+09,  ...,  6.9452e-01,
           -1.0000e+09, -1.0000e+09],
          [ 2.7665e-01,  2.9299e+00, -1.0000e+09,  ..., -1.2391e+00,
           -1.0000e+09, -1.0000e+09],
          ...,
          [ 1.2543e+00,  6.4123e-01, -1.0000e+09,  ...,  1.3318e+00,
           -1.0000e+09, -1.0000e+09],
          [ 1.6998e-01,  2.5506e-01, -1.0000e+09,  ...,  1.8486e+00,
           -1.0000e+09, -1.0000e+09],
          [ 1.5119e+00,  7.1501e-01, -1.0000e+09,  ..., -4.5649e-01,
           -1.0000e+09, -1.0000e+09]],

         [[-2.1752e-01,  2.2028e+00, -1.0000e+09,  ...,  4.6284e-01,
           -1.0000e+09, -1.0000e+09],
          [ 1.7189e-01,  1.7562e-01, -1.0000e+09,  ...,  8.8191e-01,
           -1.0000e+09, -1.0000e+09],
          [-3.0801e-01, -8.1665e-01, -1.0000e+09,  ..., -1.5282e+00,
           -1.0000e+09, -1.0000e+09],
          ...,
     

In [49]:
from tqdm import tqdm
from time import sleep

In [50]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

Processing d: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.94it/s]


In [55]:
torch.mean(torch.tensor([1,2,3]))

RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Long

In [56]:
a = [1,2,3]
sum(a)/len(a)

2.0

In [2]:
%cd ..

/home/steven/proj/lifematrix/TransformerLM


In [6]:
!pip install dotteddict

  Using cached dotteddict-2016.3.11.tar.gz (3.1 kB)
  Created wheel for dotteddict: filename=dotteddict-2016.3.11-py2.py3-none-any.whl size=3293 sha256=52a9adc8d274e9dd84300503f146a71ef68cf60f60c4156722f6097f0af972b9
  Stored in directory: /home/steven/.cache/pip/wheels/f4/5a/14/21c36cf5a3c1996f647e7d219cf05a93746173f7d811061438
Successfully built dotteddict


In [7]:
import torch
from dotteddict import dotteddict

In [8]:
state = torch.load("checkpoints/20240118.pt")

In [9]:
state.keys()

dict_keys(['model_config', 'model', 'lanmgr', 'lans'])

In [11]:
state['model_config'].d_ff

512

In [14]:
state['lanmgr']['vocabs']['en']['idx_map']

{'<unk>': 0,
 '<pad>': 1,
 '<bos>': 2,
 '<eos>': 3,
 'a': 4,
 '.': 5,
 'A': 6,
 'in': 7,
 'the': 8,
 'on': 9,
 'is': 10,
 'and': 11,
 'man': 12,
 'of': 13,
 'with': 14,
 ',': 15,
 'woman': 16,
 'are': 17,
 'to': 18,
 'Two': 19,
 'at': 20,
 'wearing': 21,
 'people': 22,
 'shirt': 23,
 'white': 24,
 'young': 25,
 'black': 26,
 'his': 27,
 'an': 28,
 'while': 29,
 'blue': 30,
 'red': 31,
 'sitting': 32,
 'girl': 33,
 'dog': 34,
 'boy': 35,
 'men': 36,
 'standing': 37,
 'playing': 38,
 'group': 39,
 'street': 40,
 'down': 41,
 'walking': 42,
 '-': 43,
 'front': 44,
 'her': 45,
 'holding': 46,
 'water': 47,
 'by': 48,
 'The': 49,
 'up': 50,
 'green': 51,
 'women': 52,
 'An': 53,
 'one': 54,
 'for': 55,
 'looking': 56,
 'outside': 57,
 'child': 58,
 'Three': 59,
 'as': 60,
 'little': 61,
 'large': 62,
 'through': 63,
 'yellow': 64,
 'brown': 65,
 'two': 66,
 'from': 67,
 'hat': 68,
 'ball': 69,
 'their': 70,
 'into': 71,
 'person': 72,
 'children': 73,
 'next': 74,
 'other': 75,
 'dressed': 